In [1]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from src.data.Dataset import create_3d_volumes_from_4d_files, create_4d_volumes_from_4d_files
from src.utils.utils_io import Console_and_file_logger, ensure_dir
Console_and_file_logger('map_patients_of_two_exports')

search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


Using TensorFlow backend.
2020-05-29 14:22:30,505 INFO -------------------- Start --------------------
2020-05-29 14:22:30,506 INFO Working directory: /mnt/data/git/cardio.
2020-05-29 14:22:30,506 INFO Log file: ./logs/map_patients_of_two_exports.log
2020-05-29 14:22:30,507 INFO Log level for console: INFO


- get a list of all SAX first volumes
- get a list of all SAX second volumes

In [13]:
sax1_volumes = sorted(glob.glob('data/raw/GCN/4D/**/images/*clean.nrrd'))
print('SAX 1 volumes: {}'.format(len(sax1_volumes)))
print(sax1_volumes[:2])
sax2_volumes = sorted(glob.glob('data/raw/gcn_05_2020/SAx/*clean.nrrd'))
print('SAX 2 volumes: {}'.format(len(sax2_volumes)))
print(sax2_volumes[:2])

SAX 1 volumes: 209
['data/raw/GCN/4D/test/images/0019-1EMMCVKN-2006-06-29_volume_clean.nrrd', 'data/raw/GCN/4D/test/images/0051-2HQVK81W-2005-03-11_volume_clean.nrrd']
SAX 2 volumes: 279
['data/raw/gcn_05_2020/SAx/0000-02Z1L14N_2006-11-09_volume_clean.nrrd', 'data/raw/gcn_05_2020/SAx/0000-04NEJQUZ_2007-03-13_volume_clean.nrrd']


In [14]:
# extract the patient id and year from both lists
def extract_patient_year(file_name):
    return os.path.basename(file_name).split('-')[1]
sax1_patients = set(map(extract_patient_year, sax1_volumes))
sax2_patients = set(map(extract_patient_year, sax2_volumes))
sax2_patients = set(map(lambda x: x.split('_')[0], sax2_patients))
print(len(sax1_patients), len(sax2_patients))

209 279


In [15]:
print(list(sax1_patients)[:5])
print(list(sax2_patients)[:5])

['1KLZMGEG', '4A8MPQF4', 'CNT7A7NE', 'F0QP6ZJR', 'FANDNZ55']
['1KLZMGEG', '4A8MPQF4', 'F0QP6ZJR', 'FANDNZ55', '5JGEZLYC']


In [16]:
# get intersection
intersection = sax1_patients.intersection(sax2_patients)
len(intersection)

183

In [18]:
# filter ax_volumes and sax_volumes by the intersection
def filter_by_patient_intersection(file_name, split=False):
    if split:
        return extract_patient_year(file_name).split('_')[0] in intersection
    else:
        return extract_patient_year(file_name) in intersection
# we need the lambda for the fix splitting parameter, sax2 is patientid and year, need to split
sax1_volumes_filtered = list(filter(filter_by_patient_intersection, sax1_volumes))
sax2_volumes_filtered = list(filter(lambda x: filter_by_patient_intersection(x,True), sax2_volumes))
print(len(sax1_volumes_filtered), len(sax2_volumes_filtered))

183 183


In [22]:
# create file-lists for the mask
sax1_masks_filtered = list(map(lambda x:x.replace('clean', 'mask').replace('images', 'masks'), sax1_volumes_filtered))
sax2_masks_filtered = list(map(lambda x:x.replace('clean', 'mask'), sax2_volumes_filtered))

In [45]:
# copy 4D images/masks to new directory
# define a helper copy function
def copy_volumes(files, destination_path):
    """
    Copy all files named in files to the destination path
    files: list of full file paths as strings
    destination_path: root destination folder as string
    """
    import shutil
    logging.info('Start to copy: {} files'.format(len(files)))
    # ensure directory exist
    ensure_dir(destination_path)
    for file in files:
        logging.info('copy file: {} to {}'.format(file, destination_path))
        shutil.copy2(file, destination_path)
    # get all nrrd files in the destination directory
    copied_files = glob.glob(os.path.join(destination_path, '*.nrrd'))
    logging.info('Copy finished, found {} files in destination path {}'.format(len(copied_files), destination_path))

export_path = 'data/raw/gcn_05_2020_sax_interobserver'
copy_volumes(sax1_volumes_filtered, os.path.join(export_path, 'SAX1'))
copy_volumes(sax1_masks_filtered, os.path.join(export_path, 'SAX1'))
copy_volumes(sax2_volumes_filtered, os.path.join(export_path, 'SAX2'))
copy_volumes(sax2_masks_filtered, os.path.join(export_path, 'SAX2'))

2020-05-29 15:11:26,464 INFO Start to copy: 183 files
2020-05-29 15:11:26,465 INFO copy file: data/raw/GCN/4D/test/images/0019-1EMMCVKN-2006-06-29_volume_clean.nrrd to data/raw/gcn_05_2020_sax_interobserver/SAX1
2020-05-29 15:11:26,498 INFO copy file: data/raw/GCN/4D/test/images/0051-2HQVK81W-2005-03-11_volume_clean.nrrd to data/raw/gcn_05_2020_sax_interobserver/SAX1
2020-05-29 15:11:26,594 INFO copy file: data/raw/GCN/4D/test/images/0066-2Y9U5LLC-2006-07-24_volume_clean.nrrd to data/raw/gcn_05_2020_sax_interobserver/SAX1
2020-05-29 15:11:26,671 INFO copy file: data/raw/GCN/4D/test/images/0068-3E8U83K5-2006-04-01_volume_clean.nrrd to data/raw/gcn_05_2020_sax_interobserver/SAX1
2020-05-29 15:11:26,792 INFO copy file: data/raw/GCN/4D/test/images/0107-4Q0TDELR-2007-03-29_volume_clean.nrrd to data/raw/gcn_05_2020_sax_interobserver/SAX1
2020-05-29 15:11:26,809 INFO copy file: data/raw/GCN/4D/test/images/0129-5YX47QXL-2006-10-13_volume_clean.nrrd to data/raw/gcn_05_2020_sax_interobserver/SAX

In [46]:
files = sorted(glob.glob('data/raw/gcn_05_2020_sax_interobserver/SAX1/*.nrrd'))

In [47]:
def replace_symbol_at_index(string, index=13, replace='_'):
    return '{}/{}{}{}'.format(os.path.dirname(string),os.path.basename(string)[:13],'_',os.path.basename(string)[14:])
files_new = list(map(replace_symbol_at_index, files))

In [48]:
files_new[:2]

['data/raw/gcn_05_2020_sax_interobserver/SAX1/0003-04NEJQUZ_2007-03-13_volume_clean.nrrd',
 'data/raw/gcn_05_2020_sax_interobserver/SAX1/0003-04NEJQUZ_2007-03-13_volume_mask.nrrd']

In [49]:
def rename_wrapper(tup):
    import os
    src, dest = tup
    os.rename(src,dest)
    return True
list(map(rename_wrapper, zip(files, files_new)))

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,